# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
df = pd.read_csv('temperatures.csv')
df = df.iloc[:, 1:]
df.head()

,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed
0,Avarua,-159.7750,-21.2078,86.00,70,20,6.91
1,Rikitea,-134.9692,-23.1203,80.24,76,55,16.55
2,Busselton,115.3333,-33.6500,86.77,55,99,1.99
3,Butaritari,172.7902,3.0707,79.92,79,96,19.15
4,Vila,-8.1588,42.0304,48.99,68,100,1.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [304]:
gmaps.configure(api_key=g_key)

locations = df[["Latitude", "Longitud"]]
humidity = df["Humidity"]



In [305]:
humidity.max()

100

In [306]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
#base_url = "https://maps.googleapis.com/maps/api/geocode/json?" 

#params = {"latlng":(-21.2078, -159.7750), "key": g_key}
#response = requests.get(base_url, params = params).json()

Narrowing down the DataFrame the ideal weather condition:

-A max temperature lower than 80 degrees but higher than 70.

-Wind speed less than 10 mph.

-Zero cloudiness.

In [69]:
ideal_df = df[(df['Temperature']<80) & (df['Temperature']>70) & (df['Wind_speed']<10) & (df['Cloudiness']==0)]

In [71]:
ideal_df

,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed
137,Babu,111.5167,24.4167,73.26,53,0,1.05
214,Umluj,37.2685,25.0213,73.67,64,0,4.99
264,Tigre,-58.5796,-34.4260,72.18,88,0,5.75
281,Solenzo,-4.0833,12.1833,76.98,17,0,3.33
293,Filingué,3.3168,14.3521,73.15,19,0,7.23
315,Hong Kong,114.1577,22.2855,70.30,53,0,3.44
321,Ipanema,-51.2244,-30.1391,72.34,83,0,8.05
374,Moundou,16.0833,8.5667,71.60,22,0,3.53
451,La Reforma,-91.0167,18.1000,72.28,80,0,3.33
460,Ouallam,2.0866,14.3188,75.11,17,0,8.81


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [129]:
hotel_df = ideal_df

In [242]:
hotel_df

,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed
0,Babu,111.5167,24.4167,73.26,53,0,1.05
1,Umluj,37.2685,25.0213,73.67,64,0,4.99
2,Tigre,-58.5796,-34.4260,72.18,88,0,5.75
3,Solenzo,-4.0833,12.1833,76.98,17,0,3.33
4,Filingué,3.3168,14.3521,73.15,19,0,7.23
5,Hong Kong,114.1577,22.2855,70.30,53,0,3.44
6,Ipanema,-51.2244,-30.1391,72.34,83,0,8.05
7,Moundou,16.0833,8.5667,71.60,22,0,3.53
8,La Reforma,-91.0167,18.1000,72.28,80,0,3.33
9,Ouallam,2.0866,14.3188,75.11,17,0,8.81


In [198]:
lats = []
lngs = []
lat_lng = []

for row in hotel_df.iterrows():
    
    lats.append(row[1]['Latitude'])
    lngs.append(row[1]['Longitud'])

lat_lng = list(zip(lats, lngs))

In [244]:
len(lat_lng)

13

In [294]:
names = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
#target_coordinates = []
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary

        
for target in lat_lng:
    params = {
    "location": f"{target[0]}, {target[1]}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key }
    try: 
        response = requests.get(base_url, params=params).json()
        names.append(response['results'][0]['name'])
    except: 
        names.append(None)
    

In [295]:
names

['Xin Hai International Holiday Hotel',
 'HP Red Sea Hotel',
 'Villa San Gimignano',
 'Solenzo',
 None,
 'The Harbourview',
 None,
 'Club du Logone',
 None,
 None,
 'Hotel Terrado Cavancha',
 'Golden Tulip Porto Vitória',
 'InterTower Hotel']

In [296]:
hotel_df['Hotel Name'] = names
hotel_df

<ipython-input-296-0dbec5bcd668>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = names


,City,Longitud,Latitude,Temperature,Humidity,Cloudiness,Wind_speed,Hotel Name
0,Babu,111.5167,24.4167,73.26,53,0,1.05,Xin Hai International Holiday Hotel
1,Umluj,37.2685,25.0213,73.67,64,0,4.99,HP Red Sea Hotel
2,Tigre,-58.5796,-34.4260,72.18,88,0,5.75,Villa San Gimignano
3,Solenzo,-4.0833,12.1833,76.98,17,0,3.33,Solenzo
4,Filingué,3.3168,14.3521,73.15,19,0,7.23,None
5,Hong Kong,114.1577,22.2855,70.30,53,0,3.44,The Harbourview
6,Ipanema,-51.2244,-30.1391,72.34,83,0,8.05,None
7,Moundou,16.0833,8.5667,71.60,22,0,3.53,Club du Logone
8,La Reforma,-91.0167,18.1000,72.28,80,0,3.33,None
9,Ouallam,2.0866,14.3188,75.11,17,0,8.81,None


In [308]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_1 = hotel_df[["Latitude", "Longitud"]]

In [ ]:
<dt>Country</dt><dd>{Country}</dd>

In [310]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
symbol_layer = gmaps.symbol_layer(
    locations_1, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info)

fig.add_layer(symbol_layer)
fig.add_layer(heat_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

Here we can see the places with ideal weather to go for vacations
Some of them have no nearby hotel (less than 5km)